In [1]:
%reload_ext autoreload
%autoreload 2

In [21]:
import pypi_api

import requests
import tarfile
import os

In [3]:
packages = pypi_api.get_packages()

In [23]:
# collect a subet of package metadata

package_metadata = {}
for package in packages[1000:1100]:
    package_metadata[package] = pypi_api.get_package_metadata(package)

In [33]:
def download_and_extract(package_metadata, target_dir):

    # Extract package info
    name = package_metadata['info']['name']
    version = package_metadata['info']['version']

    # Ensure the directory exists
    os.makedirs(target_dir, exist_ok=True)

    # Find the source distribution URL (.tar.gz)
    for release in package_metadata['releases'][version]:
        if release['packagetype'] == 'sdist':
            url = release['url']
            break
    else:
        print(f" - No source distribution found for {name}-{version}")
        return

    # Download the source distribution
    response = requests.get(url)
    if response.status_code == 200:
        tar_path = os.path.join(target_dir, f"{name}.tar.gz")
        with open(tar_path, 'wb') as file:
            file.write(response.content)

        if os.path.exists(tar_path) and os.path.getsize(tar_path) > 0:
            try:
                with tarfile.open(tar_path) as tar:
                    tar.extractall(path=target_dir)
            except tarfile.ReadError:
                print(f" - Failed to read {tar_path}. It may not be a tar archive.")
        else:
            print(f" - {tar_path} does not exist or is empty.")
        os.remove(tar_path)
    else:
        print(f" - Failed to download {url}")

In [36]:
for name, metadata in package_metadata.items():
    if metadata.get('msg') != f'Failed to retrieve {name}':
        print(f"Downloading {name}")
        download_and_extract(metadata, 'downloads')
    else:
        print(f"Skipping {name}")

 - No source distribution found for abb-motion-program-exec-0.7.0
 - No source distribution found for abb-robot-client-0.3.0
 - No source distribution found for abb-robotraconteur-driver-hmp-0.2.0
 - No source distribution found for abbrs-0.0.15
 - No source distribution found for abbyInPython-1.0.0
 - No source distribution found for abc-0329-1687508180
 - No source distribution found for abc-0329-linux-1627139781
 - Failed to read downloads/abc1.tar.gz. It may not be a tar archive.
 - Failed to read downloads/ab_calc.tar.gz. It may not be a tar archive.
 - No source distribution found for abc-analysis-0.1.22
 - No source distribution found for abcd-0329-1645514386
 - Failed to read downloads/abcdefg.tar.gz. It may not be a tar archive.
Skipping abcdmini
 - No source distribution found for abcgh-1.0.0
 - No source distribution found for abc-matlab.py-0.1
Skipping abcmodel
Skipping abcp-api-beta
 - No source distribution found for abc-property-1.0
 - No source distribution found for ab